Reformat published datasets for CysteineomeDB. Categories: Dataset Identified, Reactive, and Ligandable. Note: Annotations from authors were used to determine "hyperreactivity" (R < 2).

# Setup Environment

In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib
import numpy as np
import math
from matplotlib.pyplot import figure
import Bio
from Bio import SeqIO
from functools import reduce
import seaborn as sns
from statistics import mean

In [ ]:
cd = os.getcwd()
cd

In [ ]:
date = '220919'

In [ ]:
path_data = os.path.join(os.getcwd(), 'results')
if not os.path.exists(path_data):
    os.makedirs(path_data)

In [ ]:
# merge identifiers from all csvs

def get_new_df(dfs, dataset, col1, col2, cys):
    new_df = pd.concat(dfs)
    
    new_df = new_df[new_df[col1].str.contains("contaminant") == False]
    
    if dataset == 'kuljanin_gygi':
        new_df['proteinid'] = new_df[col1].map(lambda x: str(x).split('|')[1])
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x))
    elif dataset == 'weerapana_cravatt':
        new_df['proteinid'] = new_df[col1].map(lambda x: str(x))
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x))
    elif dataset == 'backus_cravatt':
        new_df['proteinid'] = new_df['Identifier'].map(lambda x: str(x).split('_')[0])
        new_df['resid'] = new_df['Identifier'].map(lambda x: str(x).split('_')[-1]) 
    elif dataset == 'yan_backus':
        new_df['proteinid'] = new_df[col1]
        new_df['resid'] = new_df['identifier'].map(lambda x: 'C' + str(x).split('_')[-1]) 
    elif dataset == 'yang_wang':
        new_df['proteinid'] = new_df[col1]
        new_df['resid'] = new_df[col2].map(lambda x: 'C' + str(x))         
    else:
        new_df = new_df.rename(columns = {col1: 'proteinid', col2: 'resid'})
        
    new_df['cysteineid'] = new_df['proteinid'] + '_' + new_df['resid'].astype(str)
    new_df['dataset'] = dataset
    new_df['identified'] = 1
    new_df['identified_datasets'] = dataset
    
    if cys == True:
        new_df['level'] = 'cysteine'
        new_df = new_df[['level', 'cysteineid', 'proteinid', 'dataset', 'identified', 'identified_datasets']]
    else:
        new_df['level'] = 'protein'
        new_df = new_df[['level', 'proteinid', 'dataset', 'identified', 'identified_datasets']]
    new_df = new_df.drop_duplicates()
    
    return new_df

In [ ]:
# create protein identifiers
# UniProtKB

def get_pro_uniprot_identifier(master, df, dataset, category, category_datasets, col1, col2):
    if dataset == 'weerapana_cravatt':
        df['proteinid'] = df[col1].map(lambda x: str(x))
    elif dataset == 'kuljanin_gygi':
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df['proteinid'] = df[col1].map(lambda x: str(x).split('|')[1])
    elif dataset == 'backus_cravatt':
        df['proteinid'] = df['Identifier'].map(lambda x: str(x).split('_')[0])
        df = df[['proteinid']]
        df = df.drop_duplicates()
    elif dataset == 'yang_wang':
        df['proteinid'] = df[col1]
    else:
        df = df[[col1]]
        df = df.drop_duplicates()
        df = df.rename(columns = {col1: 'proteinid'})

    df_ids = list(df['proteinid'].unique())
    
    master[category] = np.where(master['proteinid'].isin(df_ids), 1, 0)
    category_df = master[master[category] == 1]
    category_df[category  + '_datasets'] = [category_datasets] * category_df.shape[0]
    non_category_df = master[master[category] == 0]
    
    new_df = pd.concat([category_df, non_category_df])
    
    return new_df

In [ ]:
# create cysteine identifiers
# UniProtKB_C#

def get_cys_uniprot_identifier(master, df, dataset, category, category_datasets, col1, col2):
    if dataset == 'weerapana_cravatt':
        df['proteinid'] = df[col1].map(lambda x: str(x))
        df['resid'] = df[col2].map(lambda x: 'C' + str(x))
    elif dataset == 'kuljanin_gygi':
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df['proteinid'] = df[col1].map(lambda x: str(x).split('|')[1])
        df['resid'] = df[col2].map(lambda x: 'C' + str(x))
    elif dataset == 'backus_cravatt':
        df['proteinid'] = df['Identifier'].map(lambda x: str(x).split('_')[0])
        df['resid'] = df['Identifier'].map(lambda x: str(x).split('_')[-1])  
        df = df[['proteinid', 'resid']]
        df = df.drop_duplicates()
    elif dataset == 'yang_wang':
        df['proteinid'] = df[col1]
        df['resid'] = df[col2].map(lambda x: 'C' + str(x)) 
    else:
        df = df[[col1, col2]]
        df = df.drop_duplicates()
        df = df.rename(columns = {col1: 'proteinid', col2: 'resid'})
        
    df['cysteineid'] = df['proteinid'] + '_' + df['resid'].astype(str)
    df_ids = list(df['cysteineid'].unique())
    
    master[category] = np.where(master['cysteineid'].isin(df_ids), 1, 0)
    category_df = master[master[category] == 1]
    category_df[category  + '_datasets'] = [category_datasets] * category_df.shape[0]
    non_category_df = master[master[category] == 0]
    
    new_df = pd.concat([category_df, non_category_df])
    
    return new_df

In [ ]:
def list_to_string(lst, symbol):
    return (symbol.join([str(elem) for elem in lst]))

In [ ]:
# create cysdb reactivity bins

def get_reactivity_bin(df, mean):
    reactivity_labels = []
    
    for index, row in df.iterrows():
        ratio = row[mean]
        if ratio < 2:
            reactivity_labels.append('High')
        elif (ratio > 2) & (ratio <= 5):
            reactivity_labels.append('Medium')
        else:
            reactivity_labels.append('Low')
    return reactivity_labels

# Extract Experimental Data

# Read Quantifying Functional Cysteines Data

In [ ]:
os.chdir(cd)
os.chdir('Functional')

In [ ]:
df_function = pd.read_csv('nature_2010.csv')
df_function = df_function[df_function['Identifier'].str.contains('Reverse') == False]
df_function = df_function[df_function['Identifier'].str.contains('contaminant') == False]

In [ ]:
weerapana_reactivity_df = df_function[['Identifier', 'Average']]

In [ ]:
weerapana_reactivity_df = weerapana_reactivity_df.rename(columns = {'Average': 'weerapana_mean', 'Identifier': 'cysteineid'})

In [ ]:
weerapana_reactivity_df.shape[0], len(weerapana_reactivity_df['cysteineid'].unique())

In [ ]:
weerapana_reactivity_df['weerapana_reactivity_category'] = get_reactivity_bin(weerapana_reactivity_df, 'weerapana_mean')

In [ ]:
os.chdir('../')
os.chdir('results')
weerapana_reactivity_df.to_csv('weerapana_reactive_dataset.csv', index = False)

# Read Mapping Data

In [ ]:
os.chdir(cd)
os.chdir('Mapping')

In [ ]:
df_mapping = pd.read_excel('msb20209840-sup-0020-datasetev18.xlsx', sheet_name='Sheet2')

In [ ]:
palafox_reactivity_df = df_mapping[['pos.ID', 'Mean']]

In [ ]:
palafox_reactivity_df = palafox_reactivity_df.rename(columns = {'pos.ID': 'cysteineid', 'Mean': 'palafox_mean'
                                                               })

In [ ]:
palafox_reactivity_df.shape[0], len(palafox_reactivity_df['cysteineid'].unique())

In [ ]:
palafox_reactivity_df['palafox_reactivity_category'] = get_reactivity_bin(palafox_reactivity_df, 'palafox_mean')

In [ ]:
os.chdir('../')
os.chdir('results')
palafox_reactivity_df.to_csv('palafox_reactive_dataset.csv', index = False)

# Read T-cell Data

In [ ]:
os.chdir(cd)
os.chdir('Tcell')

## Read reactivity data

In [ ]:
df_tcell_reactivity = pd.read_excel('NIHMS1616434-supplement-mmc4.xlsx', sheet_name='Table S6_Master Table', header = [5])

In [ ]:
df_tcell_reactivity['Residue'] = df_tcell_reactivity['Residues'].str.split(',')
df_tcell_reactivity_split = df_tcell_reactivity.explode('Residue')

In [ ]:
vinogradova_df = get_new_df([df_tcell_reactivity_split], 'vinogradova_cravatt', 'Uniprot', 'Residue', True)

In [ ]:
df_tcell_reactive_control = df_tcell_reactivity_split[df_tcell_reactivity_split['control'].isna() == False]
df_tcell_reactive_activated = df_tcell_reactivity_split[df_tcell_reactivity_split['activated'].isna() == False]

In [ ]:
df_tcell_reactive_control['cysteineid'] = df_tcell_reactive_control['Uniprot'].str.strip() + '_' + df_tcell_reactive_control['Residue'].str.strip()

## Average duplicates 

In [ ]:
dup_vinogradova_reactivity_df = df_tcell_reactive_control[['cysteineid', 'control']]

In [ ]:
dup_vinogradova_reactivity_df = dup_vinogradova_reactivity_df.rename(columns = {'control': 'vinogradova_mean'
                                                                       })

In [ ]:
dup_vinogradova_reactivity_df = dup_vinogradova_reactivity_df.drop_duplicates()

In [ ]:
ids = []
duplicate_ids = []
for index, row in dup_vinogradova_reactivity_df.iterrows():
    if row['cysteineid'] not in ids:
        ids.append(row['cysteineid'])
    else:
        duplicate_ids.append(row['cysteineid'])

In [ ]:
dup_vinogradova_reactivity_df['duplicate'] = np.where(dup_vinogradova_reactivity_df['cysteineid'].isin(duplicate_ids), 'yes', 'no')

In [ ]:
no_dups_vinogradova_reactivity_df = dup_vinogradova_reactivity_df[dup_vinogradova_reactivity_df['duplicate'] == 'no']

In [ ]:
no_dups_vinogradova_reactivity_df = no_dups_vinogradova_reactivity_df.drop(columns = ['duplicate'])

In [ ]:
dups_vinogradova_reactivity_df = dup_vinogradova_reactivity_df[dup_vinogradova_reactivity_df['duplicate'] == 'yes']

In [ ]:
def get_duplicate_means(df):
    cysteines = {}
    
    for index, row in df.iterrows():
        if row['cysteineid'] not in cysteines.keys():
            cysteines[row['cysteineid']] = [row['vinogradova_mean']]
        else:
            cysteines[row['cysteineid']] = cysteines[row['cysteineid']] + [row['vinogradova_mean']]
            
    dup_df = pd.DataFrame()
    
    dup_df['cysteineid'] = list(cysteines.keys())
    dup_df['vinogradova_means'] = list(cysteines.values())
    
    new_means = []
    for index, row in dup_df.iterrows():
        means = row['vinogradova_means']
        new_mean = mean(means)
        new_means.append(new_mean)
        
    dup_df['vinogradova_mean'] = new_means
    
    return dup_df[['cysteineid', 'vinogradova_mean']]

In [ ]:
dups_vinogradova_reactivity_df = get_duplicate_means(dups_vinogradova_reactivity_df)

In [ ]:
dups_vinogradova_reactivity_df

In [ ]:
vinogradova_reactivity_df = pd.concat([no_dups_vinogradova_reactivity_df, dups_vinogradova_reactivity_df])

In [ ]:
vinogradova_reactivity_df.shape[0], len(vinogradova_reactivity_df['cysteineid'].unique())

In [ ]:
vinogradova_reactivity_df['vinogradova_reactivity_category'] = get_reactivity_bin(vinogradova_reactivity_df, 'vinogradova_mean')

In [ ]:
os.chdir('../')
os.chdir('results')
vinogradova_reactivity_df.to_csv('vinogradova_reactive_dataset.csv', index = False)

# Aggregate Reactivity Data

In [ ]:
weerapana_reactivity_df = pd.read_csv('weerapana_reactive_dataset.csv')
palafox_reactivity_df = pd.read_csv('palafox_reactive_dataset.csv')
vinogradova_reactivity_df = pd.read_csv('vinogradova_reactive_dataset.csv')

In [ ]:
w_c_ids = list(weerapana_reactivity_df['cysteineid'].unique())
p_c_ids = list(palafox_reactivity_df['cysteineid'].unique())
v_c_ids = list(vinogradova_reactivity_df['cysteineid'].unique())

In [ ]:
c_ids = list(set(w_c_ids + p_c_ids + v_c_ids))

In [ ]:
len(c_ids)

In [ ]:
reactivity_df = pd.DataFrame()
reactivity_df['cysteineid'] = c_ids

In [ ]:
reactivity_df = pd.merge(reactivity_df, weerapana_reactivity_df, on = 'cysteineid', how = 'left')

In [ ]:
reactivity_df = pd.merge(reactivity_df, palafox_reactivity_df, on = 'cysteineid', how = 'left')

In [ ]:
reactivity_df = pd.merge(reactivity_df, vinogradova_reactivity_df, on = 'cysteineid', how = 'left')

# Make CysDB median - average of averages

In [ ]:
def get_dataset_count(df):
    
    df = df.fillna(0)
    
    counts = []
    for index, row in df.iterrows():
        count = 0
        
        if (row['weerapana_mean'] != 0):
            count += 1
        if (row['palafox_mean'] != 0):
            count += 1
        if (row['vinogradova_mean'] != 0):
            count += 1
        counts.append(count)
    return counts

In [ ]:
reactivity_counts = get_dataset_count(reactivity_df)

In [ ]:
tmp_reactivity_df = reactivity_df.copy()

In [ ]:
tmp_reactivity_df['dataset_count'] = reactivity_counts

In [ ]:
dataset_medians_df = tmp_reactivity_df[[
'cysteineid',
 'weerapana_mean',
 'palafox_mean',
 'vinogradova_mean'
]]

In [ ]:
dataset_medians_df = dataset_medians_df.replace(0, np.nan)

In [ ]:
dataset_medians_df['cysdb_mean'] = dataset_medians_df.mean(skipna=True, axis = 1)

In [ ]:
dataset_medians_df['cysdb_stdev'] = dataset_medians_df.std(skipna=True, axis = 1)

In [ ]:
dataset_medians_df['cysdb_median'] = dataset_medians_df.median(skipna=True, axis = 1)

In [ ]:
dataset_medians_df['cysdb_reactivity_category'] = get_reactivity_bin(dataset_medians_df, 'cysdb_median')

In [ ]:
new_reactivity_df = dataset_medians_df.copy()

In [ ]:
# identify hyperreactive based on ratio cutoff established by Weerapana et. al., Palafox et. al. and Vinogradova et. al

def get_reactive(df):
    hyper_labels = []
    for index, row in df.iterrows():
        hyper = False
        if row['weerapana_mean'] < 2:
            hyper = True
        if row['palafox_mean'] < 2:
            hyper = True
        if row['vinogradova_mean'] < 2:
            hyper = True
        if hyper == True:
            hyper_labels.append('yes')
        else:
            hyper_labels.append(np.nan)
        
    return hyper_labels

In [ ]:
hyper_labels = get_reactive(new_reactivity_df)

In [ ]:
new_reactivity_df['hyperreactive'] = hyper_labels

In [ ]:
new_reactivity_df['hyperreactive'].value_counts()

In [ ]:
new_reactivity_df['proteinid'] = new_reactivity_df['cysteineid'].map(lambda x: str(x).split('_')[0])

In [ ]:
new_reactivity_df['resid'] = new_reactivity_df['cysteineid'].map(lambda x: str(x).split('_C')[1])

In [ ]:
new_reactivity_df = new_reactivity_df[[
 'proteinid',
 'cysteineid',
 'resid',
 'weerapana_mean',
 'palafox_mean',
 'vinogradova_mean',
 'cysdb_mean',
 'cysdb_stdev',
 'cysdb_median',
 'cysdb_reactivity_category',
 'hyperreactive'
]]

In [ ]:
new_reactivity_df.to_csv('cysteineomedb_reactive_dataset.csv', index = False)